# Updated notebook with the new Lavision library


Lavision released a new Python library to read their files, called lvreader, read below: 

"
LaVision's Python library lvreader got a new version 1.2.0 and extended its feature list by dedicated read/write functionalities for particle data sets. This makes it possible to further analyze your Shake-the-Box or PTV data with your own Python post-processing scripts. Whether you are interested in whole particle distributions or single particle tracks from your experimental data, lvreader lets you directly access DaVis set files without the need of a further export step.

Using the particle write functions, it is possible to import particle data from other sources and use DaVis' advanced processing operations, such as binning or fine-scale reconstruction, to gain further insights into your data.

If you are already working with the most recent Python version 3.10, the new release of lvreader supports it now as well. Click   https://www.lavision.de/en/downloads/software/index.php   to download lvreader 1.2.0." 

Or use the direct link https://www.lavision.de/en/download.php?id=4817




In [ ]:
# !pip install /home/user/Downloads/lvreader-1.2.0/lvreader-1.2.0-cp38-cp38-linux_x86_64.whl
from pivpy import io, pivpy
from pivpy import graphics
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import pkg_resources as pkg

import pathlib


In [ ]:
filename = pathlib.Path(pkg.resource_filename('pivpy','data'))/ "PIVMAT_jet" / "B00001.VC7"
ds = io.load_vc7(filename)

In [ ]:
ds

In [ ]:
graphics.quiver(ds.isel(t=0),arrScale=10)

In [ ]:
new = ds.piv.filterf()
graphics.quiver(new.isel(t=0),arrScale=10)

In [ ]:
new = ds.piv.filterf([3,3,0])
graphics.quiver(new.isel(t=0),arrScale=10)

In [ ]:
new.isel(t=0).plot.quiver(x='x',y='y',u='u',v='v')

In [ ]:
ds = io.create_sample_Dataset(n_frames=3,rows=5,cols=10)
ds = ds.piv.filterf([.5, .5, 0.])
ds["mag"] = np.hypot(ds["u"], ds["v"])
ds.plot.quiver(x='x',y='y',u='u',v='v',hue='mag',col='t',scale=150,cmap='RdBu')


In [ ]:
# from https://github.com/kaipak/xrsigproc

from scipy.ndimage import gaussian_filter
def _get_dims(data):
    """Get primary x-y dimensions of dataset
    """
    return ('y', 'x')

def gaussian_smooth(data, sigma = [1., 1.], mask=False, mode='reflect'):
    """Apply gaussian kernel to convolution. Uses Scipy
       gaussian_filter method.
       Parameters:
       mode (str): {‘reflect’, ‘constant’, ‘nearest’, ‘mirror’, ‘wrap’}
                   What to do at edges of matrix input. See Scipy docs
                   for details on what these do.
    """
    dims = _get_dims(data)

    sc_gaussian_nd = lambda data: gaussian_filter(data, sigma, mode=mode)

    if mask:
        data_masked = data.where(data[mask_vars[dims]])
    else:
        data_masked = data.fillna(0.)

    return xr.apply_ufunc(sc_gaussian_nd, data_masked,
                          vectorize=True,
                          dask='parallelized',
                          input_core_dims = [dims],
                          output_core_dims = [dims],
                          # output_dtypes=[data.dtype]
                        )

In [ ]:
tmp = gaussian_smooth(ds)
tmp.plot.quiver(x='x',y='y',u='u',v='v',hue='mag',col='t',scale=150,cmap='RdBu')


In [ ]:
ds = io.load_vc7(filename)
new = gaussian_smooth(ds,sigma=[2., 2.])
# new.isel(t=0).plot.quiver(x='x',y='y',u='u',v='v')
graphics.quiver(new.isel(t=0))
